Full Feature Functions

In [1]:
import cv2
import glob
import os

# 1. Import Images
def import_images(path_positif, path_negative):
    all_images = []
    labels = []

    def images(path, label):
      size_folder = os.listdir(path)
      for i in range(1, len(size_folder)+1):
          # Daftar folder yang ingin Anda proses
          folder_paths = glob.glob(path+ str(i) +"/*")
          images = []
          for files in folder_paths:
              image = cv2.imread(files)
              gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
              images.append(gray)
          all_images.append(images)
          labels.append(label)

    images(path_positif, label="Positif")
    images(path_negative, label = "Negatif")

    return all_images, labels

# 2. Extract Features
from _features.features_extraction import extract_texture
from _features._selected.features_selected import selected_extract_texture
from __utils__.convert2dataframe import convert2dataframe
import pandas as pd

def concatDF(dataframe):
    df = pd.concat([dataframe.fos, 
                    dataframe.GLCM_mean, 
                    dataframe.GLCM_four_dir,
                    dataframe.GLDS, 
                    dataframe.GLRLM
                    ], axis=1)
    return df

def extracts(data, f_selection):
    if f_selection == False:
        features= extract_texture(data, mask=None)
        dataframe = convert2dataframe(features)
        df = concatDF(dataframe)
    else:
        features= selected_extract_texture(data, mask=None)
        dataframe = convert2dataframe(features)
        df = concatDF(dataframe)
    return df

# 3. Preprocess
from sklearn.preprocessing import StandardScaler

def preprocess(dataframe):
    sc = StandardScaler()
    scaler = sc.fit(dataframe)
    data_prepro= scaler.transform(dataframe)

    return data_prepro

# 4. Testing
import pickle

def testing(path_model, data):
    clf = pickle.load(open(path_model, 'rb'))
    predictions = clf.predict(data)
    print(predictions)
    ans = sum(predictions)

    result_predictions = []

    if ans < 3:
        print("Negatif")
        result_predictions.append("Negatif")
    else:
        print("Positif")
        result_predictions.append("Positif")

    return result_predictions

# 5. Classification
import time

def execution(path_1, path_0, path_model, features_selection = False):
  result_predictions = []
  execution_times = []
  images, labels = import_images(path_1, path_0)
  
  # for i in range(0, 2):
  if features_selection == False:
    print("Used Full Features Model")
    for i in range(0, len(images)):
        start_time = time.time()
        features = extracts(images[i], features_selection)
        data_prepro = preprocess(features)
        result_predict = testing(path_model, data_prepro)
        result_predictions.append(result_predict)
        end_time = time.time()
        execution_time = end_time - start_time
        execution_times.append(execution_time)
  else:
    print("Used Selected Features Model")
    for i in range(0, len(images)):
        start_time = time.time()
        features = extracts(images[i], features_selection)
        data_prepro =preprocess(features)
        result_predict = testing(path_model, data_prepro)
        result_predictions.append(result_predict)
        end_time = time.time()
        execution_time = end_time - start_time
        execution_times.append(execution_time)

  return labels, result_predictions, execution_times

def ConvertResult2DF(labels, predictions, times):
   df1 = pd.DataFrame(labels, columns=["Aktual Label"])
   df2 = pd.DataFrame(predictions, columns=["Hasil Klasifikasi"])
   df3 = pd.DataFrame(times, columns=["Waktu Eksekusi"])
   result = pd.concat([df1, df2, df3], axis= 1)

   return result

In [2]:
full_features_model = r"Save_model\Full Features.pkl"
selected_features_model = r"Save_model\Selected Features.pkl"

path_positif = r'Images\Groundtruth\Positif/'
path_negatif = r'Images\Groundtruth\Negatif/'

Full Features

In [3]:
f_labels, f_predict, f_execution_times = execution(path_positif, path_negatif, full_features_model, features_selection = False)

Used Full Features Model
[1 0 0 0 1 1]
Positif
[0 1 0 1 1 1]
Positif
[1 0 0 0 1 1]
Positif
[0 0 1 0 1 1]
Positif
[1 1 1 1 0 1]
Positif
[1 0 1 1 1 1]
Positif
[0 0 0 0 1 0]
Negatif
[1 1 0 0 0 0]
Negatif
[1 1 0 0 1 0]
Positif
[1 1 0 0 1 1]
Positif
[1 1 0 1 1 0]
Positif
[0 1 0 1 1 1]
Positif
[0 0 0 1 1 0]
Negatif
[0 0 0 1 0 1]
Negatif
[0 0 0 1 1 1]
Positif
[0 0 0 1 1 0]
Negatif
[0 0 0 1 0 0]
Negatif
[0 0 1 0 0 1]
Negatif
[0 0 1 1 1 1]
Positif
[1 0 1 0 0 0]
Negatif
[1 0 1 0 1 0]
Positif
[0 0 0 0 0 1]
Negatif


In [4]:
f_result = ConvertResult2DF(f_labels, f_predict, f_execution_times)
f_result

,Aktual Label,Hasil Klasifikasi,Waktu Eksekusi
0,Positif,Positif,36.932261
1,Positif,Positif,17.417727
2,Positif,Positif,22.282128
3,Positif,Positif,21.002324
4,Positif,Positif,9.942910
5,Positif,Positif,14.130103
6,Positif,Negatif,25.972546
7,Positif,Negatif,23.747000
8,Positif,Positif,23.153928
9,Positif,Positif,20.555734


In [7]:
print(f_result['Waktu Eksekusi'].sum())

529.9367003440857


Selected Features

In [5]:
s_labels, s_predict, s_execution_times = execution(path_positif, path_negatif, selected_features_model, features_selection = True)
# result_predictions_0, execution_times_0 = execution(path_negatif, full_features_model, features_selection = False)

Used Selected Features Model
[1 0 0 1 1 0]
Positif
[0 1 0 1 0 0]
Negatif
[1 0 1 0 0 1]
Positif
[0 0 0 1 1 1]
Positif
[1 0 0 1 0 1]
Positif
[1 0 0 1 0 0]
Negatif
[0 0 0 1 1 1]
Positif
[1 1 1 0 0 0]
Positif
[1 1 0 1 1 0]
Positif
[1 1 0 0 1 1]
Positif
[1 1 0 0 1 0]
Positif
[1 0 0 1 1 1]
Positif
[0 1 0 1 0 0]
Negatif
[0 0 0 0 0 1]
Negatif
[0 0 0 1 1 1]
Positif
[0 0 0 1 0 1]
Negatif
[0 0 0 0 0 1]
Negatif
[1 0 1 0 0 1]
Positif
[0 0 0 1 0 1]
Negatif
[1 0 0 0 1 1]
Positif
[0 0 1 0 1 1]
Positif
[0 0 0 0 0 1]
Negatif


In [6]:
s_result = ConvertResult2DF(s_labels, s_predict, s_execution_times)
s_result

,Aktual Label,Hasil Klasifikasi,Waktu Eksekusi
0,Positif,Positif,35.543000
1,Positif,Negatif,18.705838
2,Positif,Positif,23.515413
3,Positif,Positif,22.103045
4,Positif,Positif,10.638717
5,Positif,Negatif,14.864300
6,Positif,Positif,27.284909
7,Positif,Positif,23.743466
8,Positif,Positif,23.647569
9,Positif,Positif,18.017554


In [8]:
print(s_result['Waktu Eksekusi'].sum())

527.8625688552856
